In [1]:
import ast

import pandas as pd
import os 
import os.path as osp
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import json
import ast

In [2]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, RocCurveDisplay,  average_precision_score


In [3]:
from mac2024.const import ID2LABELS as ID2LABELS
from mac2024.const import fine2coarse, COARSE_LABEL, COARSE2FINE

In [4]:
experiment_folder = r'D:\Project-mpg microgesture\human_micro_gesture_classifier\experiments'
run_name = 'mac_multi'

path_to_weights = osp.join(experiment_folder,run_name,'dataset','weights.json')
with open(path_to_weights, 'r') as json_file:
    positive_weights_dict = json.load(json_file)

        
feature_names = [entry['class'] for entry in positive_weights_dict['data']]

In [5]:
# path_to_rgb = osp.join(experiment_folder,run_name,'split_loss_fine_coarse_70','eval_29_best','raw_test_results.csv')
# path_to_densepose = osp.join(experiment_folder,run_name,'densepose_input','eval_149','raw_test_results.csv')
# path_to_gt = osp.join(experiment_folder,run_name,'dataset', 'val.csv')

path_to_rgb = osp.join(experiment_folder,run_name,'split_loss_fine_coarse_70','test','raw_test_results.csv')
path_to_densepose = osp.join(experiment_folder,run_name,'densepose_input','test_151','raw_test_results.csv')
path_to_gt = osp.join(experiment_folder,run_name,'dataset', 'test.csv')




In [6]:
fine_labels = feature_names[:52]
coarse_labels = feature_names[52:]

In [7]:
df_rgb = pd.read_csv(path_to_rgb)
df_densepose = pd.read_csv(path_to_densepose)
df_gt_raw = pd.read_csv(path_to_gt)

In [8]:
df_rgb['filenames'] = df_rgb['filenames'].apply(lambda x: x.split('.')[0])
df_densepose['filenames'] = df_densepose['filenames'].apply(lambda x: x.split('.')[0])

df_rgb.set_index('filenames', inplace=True, drop=True)
df_densepose.set_index('filenames', inplace=True, drop=True)

df_gt_raw['filenames'] = df_gt_raw['metadata'].apply(lambda x: ast.literal_eval(x)['sample_id'][:52])

In [9]:
df_gt_raw['pred'] = df_gt_raw['labels']
fine_df_gt = pd.DataFrame(np.argmax(df_gt_raw['labels'].apply(lambda x: np.array(ast.literal_eval(x))[:52]).to_list(), axis=1), index=df_gt_raw['filenames'])
fine_df_gt.columns = ['gt_label']
 

In [10]:
def softmax(x):
    e_x = np.exp(x - np.max(x))  # Subtracting the maximum for numerical stability
    return e_x / e_x.sum(axis=0)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [11]:
def extract_logits(df_in, modality, labels_in=None, logit_func = None):
    df_in_logits =  df_in[[a for a in df_in.columns if 'logit' in a]]
    if labels_in:
        df_in_logits = df_in_logits[[a for a in df_in_logits.columns if a.split('-')[1] in labels_in]]
    if logit_func:
        df_in_logits = df_in_logits.apply(softmax, axis=1)
    df_in_logits['modality'] = modality
    return df_in_logits



df_rgs_fine_logits = extract_logits(df_in=df_rgb, modality='rbg', labels_in=fine_labels, logit_func=None)
df_dp_fine_logits = extract_logits(df_in=df_densepose, modality='dp', labels_in=fine_labels, logit_func=None)


In [12]:
df1 =  df_rgs_fine_logits
df2 = df_dp_fine_logits

df1_reset = df1.reset_index()
df2_reset = df2.reset_index()

# Merge the two DataFrames on 'filename'
merged_df = pd.merge(df1_reset, df2_reset, on='filenames', suffixes=('_mod1', '_mod2'))

# Calculate the average for each pair of columns
fine_average_df = merged_df[['filenames', 'modality_mod1', 'modality_mod2']].copy()
for col in df1.columns:
    if col != 'modality':
        fine_average_df[col] = merged_df[[f"{col}_mod1", f"{col}_mod2"]].mean(axis=1)

# Optionally, rename the modality columns or drop them if not needed
# average_df['modality'] = average_df['modality_mod1'] + '_' + average_df['modality_mod2']
fine_average_df = fine_average_df.drop(columns=['modality_mod1', 'modality_mod2'])

# Set 'filename' as index again if needed
fine_average_df = fine_average_df.set_index('filenames')
fine_average_df


,logit-shaking body,logit-sitting straightly,logit-shrugging,logit-turning around,logit-rising up,logit-bowing head,logit-head up,logit-tilting head,logit-turning head,logit-nodding,...,logit-rubbing eyes,logit-touching nose,logit-touching ears,logit-covering face,logit-covering mouth,logit-pushing glasses,logit-patting legs,logit-touching legs,logit-scratching legs,logit-scratching feet
filenames,,,,,,,,,,,,,,,,,,,,,
test0000,-3.201172,-3.132202,-4.597168,-6.513672,-3.237396,-0.193359,0.741455,-1.332397,-1.312256,7.816406,...,-4.901367,-5.591797,-4.206055,-4.726562,-5.800781,-3.109863,-1.606812,-1.588501,-3.062988,-4.978516
test0001,-1.023438,-0.874268,-2.749023,-2.654785,-0.395020,-0.266113,-0.357422,0.821289,0.039062,0.692383,...,-3.897461,-3.514648,-3.503906,-4.375000,-3.014648,-2.273438,-4.628906,-1.644531,-4.496094,-4.601562
test0002,-3.303101,-5.597656,-7.650391,-6.783203,-4.116211,-1.658813,-4.298828,-0.576660,-0.369629,-1.524414,...,-5.767578,-6.236328,-3.917969,-8.042969,-7.728516,-5.646484,-0.212585,-0.135742,-3.856445,-6.667969
test0003,-2.242188,0.005371,-1.365967,-2.509277,-2.787109,0.770386,-0.031540,1.274414,1.838867,0.211914,...,0.695740,3.835938,2.304688,-2.422363,0.709621,6.646484,-0.410645,-1.337158,-2.298828,-3.438477
test0004,-2.715332,-3.670898,-6.054688,-6.187500,-3.860352,-1.808424,-3.692871,-1.157959,-1.621521,-2.213379,...,-4.059570,-3.712891,-2.991699,-7.349609,-6.060547,-4.125000,0.211914,-2.781250,-3.536133,-5.982422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test1133,2.592773,-1.036621,-2.474121,-3.524414,-1.592163,-2.587891,-0.315430,0.767929,3.478516,1.277100,...,-5.135742,-4.145508,-4.152344,-6.234375,-4.616211,-5.654297,-1.293823,0.548828,-2.913086,-5.516602
test1134,2.598633,1.573730,-0.048828,-4.152344,-2.749023,-2.997559,-1.858887,-1.135986,-1.113647,0.905762,...,-5.309570,-3.674805,-2.623535,-4.497070,-4.192383,-4.168945,-2.849121,0.801025,-1.148880,-4.513672
test1135,-2.746094,-5.100098,-6.023438,-5.869141,-4.912109,-2.383057,-4.280762,0.324585,0.128174,-2.741211,...,-6.246094,-4.627930,-5.090820,-8.326172,-7.783203,-5.846680,0.013916,-1.036465,-1.960938,-7.484375


In [13]:
fine_average_df['pred'] = np.argmax(fine_average_df.values[:,:52], axis=1)


In [14]:
fine_gt = fine_df_gt['gt_label']
fine_pred = fine_average_df['pred']

rep_dict = classification_report(fine_gt,fine_pred,output_dict=True, target_names=fine_labels)
fine_classification_rep = pd.DataFrame.from_dict(rep_dict)
fine_classification_rep = fine_classification_rep.transpose().rename(index={'accuracy':'micro avg'})
fine_classification_rep.reset_index().reset_index()

ValueError: Number of classes, 49, does not match size of target_names, 52. Try specifying the labels parameter

In [27]:
coarse_pred = np.array([fine2coarse(a) for a in fine_pred.values])

In [159]:
metadata_dict_list = df_gt_raw['metadata'].apply(lambda x: ast.literal_eval(x)).values

In [160]:
coarse_gt = [a['coarse_label_txt'] for a in metadata_dict_list] 

In [161]:
COARSE_LABEL_INV = {v:k for k,v in COARSE_LABEL.items()}


In [162]:
coarse_labels = list(COARSE_LABEL.values())

In [163]:
gt = [COARSE_LABEL_INV[a] for a in coarse_gt]
pred = coarse_pred

rep_dict = classification_report(gt, pred,output_dict=True, target_names=coarse_labels)
coarse_classification_rep = pd.DataFrame.from_dict(rep_dict)
coarse_classification_rep = coarse_classification_rep.transpose().rename(index={'accuracy':'micro avg'})
coarse_classification_rep

,precision,recall,f1-score,support
body,0.640000,0.711111,0.673684,315.000000
head,0.921209,0.832512,0.874618,2233.000000
upper limb,0.765828,0.769939,0.767878,1304.000000
lower limb,0.773333,0.808926,0.790729,717.000000
body-hand,0.613095,0.677632,0.643750,152.000000
head-hand,0.755000,0.855524,0.802125,353.000000
leg-hand,0.677419,0.779297,0.724796,512.000000
micro avg,0.800394,0.800394,0.800394,0.800394
macro avg,0.735126,0.776420,0.753940,5586.000000
weighted avg,0.808866,0.800394,0.803006,5586.000000


In [164]:
a = coarse_classification_rep.loc['micro avg', 'f1-score']
b = coarse_classification_rep.loc['macro avg', 'f1-score']
c = fine_classification_rep.loc['micro avg', 'f1-score']
d = fine_classification_rep.loc['macro avg', 'f1-score']
np.mean([a,b,c,d])

0.6727179864779893

In [29]:
fine_pred

filenames
test0000     9
test0001    30
test0002    20
test0003    47
test0004    17
            ..
test1133    36
test1134    31
test1135    17
test1136    15
test1137    22
Name: pred, Length: 1138, dtype: int64

In [28]:
coarse_pred

array([1, 3, 2, ..., 2, 2, 2])

In [121]:
filenames = fine_pred.index.values
filenames = [a+'.mp4' for a in filenames]

In [122]:
# Test preds
# Compute argmax for each set of logits
argmax_fine = fine_pred.values
argmax_coarse = coarse_pred

# Create the DataFrame with appropriate index
df_test_preds = pd.DataFrame({
    'fine': argmax_fine,
    'coarse': argmax_coarse
}, index=filenames)
df_test_preds['fine_text'] = df_test_preds['fine'].map(ID2LABELS)
df_test_preds['coarse_text'] = df_test_preds['coarse'].map(COARSE_LABEL)
df_test_preds

,fine,coarse,fine_text,coarse_text
test0000.mp4,9,1,nodding,head
test0001.mp4,30,3,retracting feet,lower limb
test0002.mp4,20,2,spreading hands,upper limb
test0003.mp4,47,5,pushing glasses,head-hand
test0004.mp4,17,2,stretching arms,upper limb
...,...,...,...,...
test1133.mp4,36,4,arms akimbo,body-hand
test1134.mp4,31,3,tiptoe,lower limb
test1135.mp4,17,2,stretching arms,upper limb
test1136.mp4,15,2,pointing oneself,upper limb


In [123]:
path_to_log_folder = r'D:\Project-mpg microgesture\human_micro_gesture_classifier\experiments\mac_multi\densepose_input\test_151'
df_test_preds.to_csv(osp.join(path_to_log_folder,'prediction_full_table.csv'))

In [124]:
df_test_preds_to_save = pd.DataFrame({
    'pred_label_1': argmax_coarse,
    'pred_label_2': argmax_fine
}, index=filenames)
df_test_preds_to_save.index.names = ['id']
# df_test_preds_to_save = df_test_preds_to_save.rename_axis(index={'': 'id'})

df_test_preds_to_save.to_csv(osp.join(path_to_log_folder,'prediction.csv'))
# df_test_preds_to_save.index.name

In [125]:
# compare preds 


In [126]:
path_pred1 = osp.join(path_to_log_folder,'prediction.csv')
path_pred2 = osp.join(r'D:\Project-mpg microgesture\human_micro_gesture_classifier\experiments\mac_multi\split_loss_fine_coarse_70\test','prediction.csv')


In [127]:
df_final_rgb = pd.read_csv(path_pred1, index_col='id')
df_final_comb = pd.read_csv(path_pred2, index_col='id')

In [140]:
df_1 = df_final_rgb[['pred_label_2']]
df_2 = df_final_comb[['pred_label_2']]

In [141]:
df_1


,pred_label_2
id,
test0000.mp4,9
test0001.mp4,30
test0002.mp4,20
test0003.mp4,47
test0004.mp4,17
...,...
test1133.mp4,36
test1134.mp4,31
test1135.mp4,17


In [142]:
df_final = pd.DataFrame(index=df_1.index)
df_final['rgb'] = df_1['pred_label_2'].astype(int).apply(lambda x: ID2LABELS[x])
df_final['comb'] = df_2['pred_label_2'].astype(int).apply(lambda x: ID2LABELS[x])
df_final['same'] = df_final['rgb'] ==df_final['comb'] 
df_final.loc[~df_final['same']]

,rgb,comb,same
id,,,
test0001.mp4,retracting feet,tiptoe,False
test0007.mp4,turning head,tilting head,False
test0012.mp4,bowing head,stretching arms,False
test0013.mp4,turning head,head up,False
test0022.mp4,touching legs,shaking body,False
...,...,...,...
test1116.mp4,clenching fist,touching legs,False
test1125.mp4,stretching arms,spreading hands,False
test1132.mp4,turning head,head up,False
